In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [ ]:
path = "final_test_dataset.csv"
df_test = pd.read_csv(path)
df_test.shape

FileNotFoundError: [Errno 2] No such file or directory: '../dataset/processed/final_test_dataset.csv'

In [ ]:
#train file
path = "../dataset/processed/final_cleaned_dataset.csv"
df_train = pd.read_csv(path)
df_train.shape

In [ ]:

punctuation_signs = list("?:!.,;")
nltk.download('punkt')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

In [ ]:
df_train['item_name'] = df_train['item_name'].str.replace("\r", " ")
df_train['item_name'] = df_train['item_name'].str.replace("\n", " ")
df_train['item_name'] = df_train['item_name'].str.replace("    ", " ")
df_train['item_name'] = df_train['item_name'].str.replace('"', '')
df_train['item_name'] = df_train['item_name'].str.lower()
for punct_sign in punctuation_signs:
  df_train['item_name'] = df_train['item_name'].str.replace(punct_sign, '')
df_train['item_name'] = df_train['item_name'].str.replace("'s", "")


df_train['bullet_points'] = df_train['bullet_points'].str.replace("\r", " ")
df_train['bullet_points'] = df_train['bullet_points'].str.replace("\n", " ")
df_train['bullet_points'] = df_train['bullet_points'].str.replace("    ", " ")
df_train['bullet_points'] = df_train['bullet_points'].str.replace('"', '')
df_train['bullet_points'] = df_train['bullet_points'].str.lower()
for punct_sign in punctuation_signs:
  df_train['bullet_points'] = df_train['bullet_points'].str.replace(punct_sign, '')
df_train['bullet_points'] = df_train['bullet_points'].str.replace("'s", "")

In [ ]:

final_cols = ["sample_id","item_name","brand_name","bullet_points","unit","price"]
df_train = df_train[final_cols]
df_train.shape

In [ ]:
df_test['item_name'] = df_test['item_name'].str.replace("\r", " ")
df_test['item_name'] = df_test['item_name'].str.replace("\n", " ")
df_test['item_name'] = df_test['item_name'].str.replace("    ", " ")
df_test['item_name'] = df_test['item_name'].str.replace('"', '')
df_test['item_name'] = df_test['item_name'].str.lower()
for punct_sign in punctuation_signs:
  df_test['item_name'] = df_test['item_name'].str.replace(punct_sign, '')
df_test['item_name'] = df_test['item_name'].str.replace("'s", "")
df_test["brand_name"]


df_test['bullet_points'] = df_test['bullet_points'].str.replace("\r", " ")
df_test['bullet_points'] = df_test['bullet_points'].str.replace("\n", " ")
df_test['bullet_points'] = df_test['bullet_points'].str.replace("    ", " ")
df_test['bullet_points'] = df_test['bullet_points'].str.replace('"', '')
df_test['bullet_points'] = df_test['bullet_points'].str.lower()
for punct_sign in punctuation_signs:
  df_test['bullet_points'] = df_test['bullet_points'].str.replace(punct_sign, '')
df_test['bullet_points'] = df_test['bullet_points'].str.replace("'s", "")

In [ ]:

final_cols = ["sample_id","item_name","brand_name","bullet_points","unit"]
df_test = df_test[final_cols]
df_test.shape

In [ ]:
df_test["item_name"].fillna("No Data", inplace = True)
df_test["brand_name"].fillna("No Data", inplace = True)
df_train["brand_name"].fillna("No Data", inplace = True)
df_test["unit"].fillna("Ounce", inplace = True)
df_train["unit"].fillna("Ounce", inplace = True)

In [ ]:
df_test.isna().sum()

In [ ]:
from sklearn.model_selection import train_test_split

# Split the training data into train/validation sets
X_temp = df_train.drop(['price'], axis=1)
y_temp = df_train["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.15, 
    random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_test.shape[0]} samples")
print(f"Test set: {df_test.shape[0]} samples")

In [ ]:

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
X_train_text = X_train['item_name'] + ' ' + X_train['bullet_points']
X_test_text = X_test['item_name'] + ' ' + X_test['bullet_points']
Main_Test = df_test['item_name'] + ' ' + df_test['bullet_points']
Main_Test_vectors_tfidf = tfidf.fit_transform(Main_Test)
print(Main_Test_vectors_tfidf.shape)
print('1')
X_train_vectors_tfidf = tfidf.fit_transform(X_train_text)
print(X_train_vectors_tfidf.shape)
print('1')
X_test_vectors_tfidf = tfidf.transform(X_test_text)
print(X_test_vectors_tfidf.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Apply dimensionality reduction
svd = TruncatedSVD(n_components=1000, random_state=42)  # Reduce to 500 features
X_train_tfidf_reduced = svd.fit_transform(X_train_vectors_tfidf)
X_test_tfidf_reduced = svd.transform(X_test_vectors_tfidf)
# Main_Test_tfidf_reduced = svd.fit_transform(Main_Test_vectors_tfidf)

print(f"Reduced train shape: {X_train_tfidf_reduced.shape}")
print(f"Reduced test shape: {X_test_tfidf_reduced.shape}")
# print(f"Reduced test shape: {Main_Test_tfidf_reduced.shape}")
print(f"Explained variance ratio: {svd.explained_variance_ratio_.sum():.4f}")


In [ ]:
lr_tfidf=LinearRegression()
lr_tfidf.fit(X_train_tfidf_reduced, y_train)


In [ ]:
import joblib

# Save the model
joblib.dump(lr_tfidf, 'linear_regression_tfidf_model.pkl')

# Save the TF-IDF vectorizer and SVD reducer too
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(svd, 'svd_reducer.pkl')

print("Model and preprocessing objects saved!")

In [ ]:
import joblib
# lr_tfidf=joblib.load('linear_regression_tfidf_model.pkl')
y_predict = lr_tfidf.predict(X_test_tfidf_reduced)
# main_predict = lr_tfidf.predict(Main_Test_tfidf_reduced)

In [ ]:
d = {"pred_price" : y_predict}
df1 = pd.DataFrame(data=d)

In [ ]:
df1 = pd.DataFrame(data=d)
print(df1)

In [ ]:
df_new = pd.DataFrame({'price': y_test.values})
df_new = pd.concat([df_new, df1.reset_index(drop=True)], axis=1)
df_new.to_csv("../dataset/submission.csv", index = False, header = True)
df_new

In [ ]:
l = ["item_name", "price","pred_price"]
df_new = df_new[l]
df_new

In [ ]:
df_new.to_csv("../dataset/dump_submission.csv", index = False, header = True)

In [ ]:

path = "../dataset/submission.csv"
df_sub = pd.read_csv(path)
df_sub.head()